In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import os

In [2]:
df_main = pd.read_excel("Data/November_11/LabHoursData.xlsx")
df_main.columns

Index(['#', 'Machine Type', 'Machine', 'Wing', 'Floor', 'Run No', 'Pcs',
       'StartDate', 'StartTime', 'StopDate', 'StopTime', 'Run %',
       'LateStartReason', 'Stop Reasons', 'Recipe', 'Run Intensity',
       'Ini Category', 'Grown Category', 'RunStatus', 'IniAvgHeight',
       'ProdHeight', 'GrownHeight', 'GrowingHours', 'BreakdownTime',
       'From To Hours', 'Actual Hours', 'RunHours', 'IniWeight', 'FinalWeight',
       'GrownWeight', 'Width', 'Length', 'Dmtr', 'Unit', 'Month', 'GrRate'],
      dtype='object')

In [3]:
df_temp = df_main[['Machine', 'Run No', 'BreakdownTime', 'LateStartReason']]
df_temp.rename(columns={"Run No":"RunNo"}, inplace=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_2188\788094034.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.rename(columns={"Run No":"RunNo"}, inplace=True)


In [4]:
df_temp = df_temp[df_temp.BreakdownTime>10]

In [5]:
df_temp["BreakdownTime"] = pd.to_numeric(df_temp["BreakdownTime"])
df_temp["Less_10_Hours"] = df_temp['BreakdownTime'] - 10

In [6]:
df_temp

,Machine,RunNo,BreakdownTime,LateStartReason,Less_10_Hours
5,CC21,CC21-108,38.33,CHILLER MAINTENANCE,28.33
10,AD15,AD15-116,16.50,HLD CLN,6.50
11,AD15,AD15-117,11.67,ON TIME,1.67
14,AD13,AD13-115,18.25,G1,8.25
15,AD13,AD13-116,17.83,HLD CLN,7.83
...,...,...,...,...,...
4758,AG48,AG48-17,13.67,ON TIME,3.67
4759,AG48,AG48-18,17.05,CD,7.05
4761,AG59,AG59-2,10.33,TEST RUN,0.33
4763,AG68,AG68-2,15.17,HLD CLN,5.17


In [6]:
len(df_temp)

2473

In [7]:
df_temp.isnull().sum()

Machine            0
RunNo              0
BreakdownTime      0
LateStartReason    1
Less_10_Hours      0
dtype: int64

In [8]:
df_temp = df_temp.dropna()

In [9]:
len(df_temp)

2472

In [10]:
df_temp.isnull().sum()

Machine            0
RunNo              0
BreakdownTime      0
LateStartReason    0
Less_10_Hours      0
dtype: int64

In [11]:
df_temp.nunique()

Machine             986
RunNo              2472
BreakdownTime       633
LateStartReason      49
Less_10_Hours       633
dtype: int64

In [12]:
df_temp["Less_10_Hours"].sum()

40467.65

In [13]:
df_temp.groupby(["LateStartReason"])["Less_10_Hours"].sum()

LateStartReason
1/2PMP (1BY1)                        51.42
ALREPRC                             185.72
ARC                                 176.56
CD                                  150.79
CHILLER MAINTENANCE                  86.25
CHL                               17350.38
CHMB CHNG                           621.34
CHMB CNT                            169.42
CNT                                  21.42
ELECTRICITY                         604.74
FS1                                   6.16
FS2                                 218.53
FS3                                   7.30
G1                                 1528.47
G2                                   67.16
G3                                    1.33
GENERAL MAINTENANCE                  31.33
HLD CHANGE                          130.26
HLD CLN                             981.78
HLM LT                             2806.99
HOLDER CLEANING                      16.63
LEAK TEST                            36.50
MGT                                  2

In [14]:
df_final = df_temp.groupby(["LateStartReason"])["Less_10_Hours"].agg(['sum','count']).reset_index()

In [15]:
df_final

,LateStartReason,sum,count
0,1/2PMP (1BY1),51.42,5
1,ALREPRC,185.72,7
2,ARC,176.56,6
3,CD,150.79,15
4,CHILLER MAINTENANCE,86.25,3
5,CHL,17350.38,299
6,CHMB CHNG,621.34,100
7,CHMB CNT,169.42,14
8,CNT,21.42,2
9,ELECTRICITY,604.74,19


In [16]:
df_final = df_final.rename(columns={'sum':"over_10_hours", 'count':'Run_count'})

In [17]:
df_final

,LateStartReason,over_10_hours,Run_count
0,1/2PMP (1BY1),51.42,5
1,ALREPRC,185.72,7
2,ARC,176.56,6
3,CD,150.79,15
4,CHILLER MAINTENANCE,86.25,3
5,CHL,17350.38,299
6,CHMB CHNG,621.34,100
7,CHMB CNT,169.42,14
8,CNT,21.42,2
9,ELECTRICITY,604.74,19


In [18]:
df_final["over_10_hours"] = pd.to_numeric(df_final["over_10_hours"])
df_final["over_10_Hours_percentage"] = round(df_final["over_10_hours"] * 100 / df_final["over_10_hours"].sum(),2)

In [19]:
df_final = df_final[["LateStartReason", "Run_count", "over_10_hours", "over_10_Hours_percentage"]]
df_final.index += 1

In [20]:
if df_final.index[0] == 0:
    df_final.index += 1

In [21]:
if os.path.exists("breakdown_time_analysis.xlsx"):
    os.remove("breakdown_time_analysis.xlsx")
    
df_final.to_excel("breakdown_time_analysis.xlsx")